### 필수과제 1
- 데이터셋을 제공해 드릴 예정
- RFECV
- SFS 
    - 두 가지 방법론을 가지고 가장 중요한 피처를 선정하는 코드와 주석으로 정리해서 공유 주세요.
    - 도메인과 함께 어떤 피처를 선택하는 것이 좋겠다 라는 결론을 부탁드립니다.
    - 데이터셋은 마지막 수업 끝나고 같이 공유하면서 공유드리겠습니다.

In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

In [62]:
df_p = pd.read_csv('student-por.csv', sep=';')
df_p['subject'] = 'portugese'
df_p.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,subject
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,4,0,11,11,portugese
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,2,9,11,11,portugese
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,6,12,13,12,portugese
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,0,14,14,14,portugese
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,0,11,13,13,portugese


In [64]:
df_m = pd.read_csv('student-mat.csv', sep=';')
df_m['subject'] = 'math'
df_m.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,subject
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,math
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,math
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,math
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,math
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,math


In [ ]:
df = pd.concat([df_p,df_m],axis=0)
df = df.reset_index(drop=True)
df

In [ ]:
df.columns

In [ ]:
df = df.drop(['address', 'famsize', 'Pstatus','guardian','reason','traveltime','schoolsup','activities', 'nursery','internet','Dalc','Walc'], axis=1)
df = pd.get_dummies(df, drop_first=True)
df.info()
 
# 'address'    학교 주소와 중복된다고 판단하여 제거
# 'traveltime' 학교 거리와 주소 중복된다고 판단하여 제거
# 'famsize'    가족구성원 여부와 성적 간 관계 없다고 판단하여 제거
# 'Pstatus'    부모의 동거 여부와 성적 간 관계 없다고 판단하여 제거
# 'guardian'   부모의 동거 여부와 유사한 항목으로 판단하여 제거
# 'reason'     진학 동기와 성적 간 관계 없다고 판단하여 제거
# 'schoolsup'  성적이 낮은 학생에게 보충교육 실시하여 피쳐가 중복된다고 판단하여 제거
# 'activities' 교내 활동 여부와 성적 간 관계 없다는 내용 확인 후 제거
# 'nursery'    간호학교 여부와 성적 간 관계 없다고 판단하여 제거
# 'internet'   인터넷 여부가 부모님의 직업과 관련있다고 판단하여 제거
# 'Dalc','Walc' 음주 여부가 성적과 관계 없다고 판단하여 제거 

In [120]:
X = df.drop('G3', axis=1)*1
y = df['G3']

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

### RFECV

In [125]:
model = LogisticRegression(max_iter=150)
cv = StratifiedKFold(3)

In [127]:
selector = RFECV(estimator = model, step=1, cv=cv)
selector =selector.fit(X,y)

In [136]:
print('최적의 수 ',selector.n_features_)
print('최적의 수 ',selector.support_)
print('최적의 수 ',selector.ranking_)

최적의 수  1
최적의 수  [False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False False False]
최적의 수  [ 2 19 12 13  9 11  5 14  6  4  3  1  8 17 23 20 24 21 26 22 10 27 18 15
 25 16  7]


In [166]:
RFECV_df = pd.DataFrame(selector.ranking_, index=X_train.columns, columns=['ranking'])
RFECV_df.sort_values(ascending=True, by='ranking')

,ranking
G2,1
age,2
G1,3
absences,4
freetime,5
health,6
subject_portugese,7
school_MS,8
failures,9
Fjob_services,10


### SFS

In [129]:
sfs_forward = SFS(model, k_features='best', forward=True, floating=False, scoring='accuracy', cv=3)
sfs_backward = SFS(model, k_features='best', forward=False, floating=True, scoring='accuracy', cv=3)
sfs_stepwise = SFS(model, k_features='best', forward=True, floating=True, scoring='accuracy', cv=3)

sfs_forward.fit(X_train, y_train)
sfs_backward.fit(X_train, y_train)
sfs_stepwise.fit(X_train, y_train)

SequentialFeatureSelector(cv=3, estimator=LogisticRegression(max_iter=150),
                          floating=True, k_features=(1, 27),
                          scoring='accuracy')

In [130]:
print('sfs_forward 피처 출력')
print(sfs_forward.k_feature_names_)

print('sfs_backward 피처 출력')
print(sfs_backward.k_feature_names_)

print('sfs_stepwise 피처 출력')
print(sfs_stepwise.k_feature_names_)

sfs_forward 피처 출력
('G2', 'Fjob_health')
sfs_backward 피처 출력
('G2', 'Fjob_health')
sfs_stepwise 피처 출력
('G2', 'Fjob_health')


### 결과 해석

- SFS 와 RFE 모두 최종 성적에 영향을 미치는 변수로 두번째 시험 성적인 **G2**를 뽑았다. 이것은 학생들에게 성적에 영향을 미치는 가장 큰 요인이 이전 성적 결과라는 것을 시사한다고 볼 수 있다.
- SFS 에서는 **Fjob_health** 라는 변수를 다음 변수로 선택하였다. 포르투갈과 인접한 스페인에서는 대학의 유무가 직업의 임금을 결정하는데 큰 영향을 미치지 않아 대학 진학률이 높지 않다고 한다. 오히려, 학생들의 학업 성적에 영향을 미치는 요인으로는 부모님의 교육수준, 직업률이 높다는 연구도 있었다. 이러한 점을 고려했을 때, 아버지의 직업(의료계)이 학생의 성적에 미치는 영향이 주요하다는 것을 알 수 있다.
- 국가마다 학생들의 성적에 영향을 미치는 요인은 상이하다. 하지만, 해당 데이터셋에서는 RFE 기준으로 성적을 제외하고 age, absences, freetime, health, subject, school, failure, Fjob의 변수가 상위 10개 변수로 나온다. 주요 피쳐가 학생들 개인 상황이나 성실도와 관련된 측도로 보았을 때, 포르투갈의 학생들은 개인의 노력(성실도)이 성적에 주요한 영향을 미친다는 것으로 해석할 수 있다. 